In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, KBinsDiscretizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

from functools import reduce

pd.set_option("display.max_columns", 500)

In [ ]:
df = pd.read_excel("/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/Unidad 1/Datasets/Cuestionario Diagnóstico 2019 - 1 (Respuestas).xlsx")

df.drop(columns=["Marca temporal", "Número de cuenta"], axis=1, inplace=True)

df.head()

,Carrera,Género,Edad 🎂,Estado civil,¿Tienes hijos?,¿Tienes alguna dificultad para? [👓 Ver],¿Tienes alguna dificultad para? [👂 Escuchar],¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física],¿Dónde vives actualmente?,¿En qué delegación o municipio?,¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱,¿Qué medio de transporte utilizas para llegar a la facultad?,¿Cuál de las siguientes opciones describe tu tipo de vivienda?,¿Con quién vives? (Puedes seleccionar más de una),¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti,¿Cuántos son hombres?,¿Cuántas son mujeres?,¿Cuánto tiempo llevas viviendo ahí? (años),¿Cuántos cuartos para dormir tiene tu vivienda?,¿Cuál es el máximo nivel de estudios de tu papá?,¿Cuál es el máximo nivel de estudios de tu mamá?,¿Con cuáles de los siguientes servicios básicos cuentas en tu hogar? Puedes seleccionar más de uno,¿Cuántos automóviles en total tienen las personas que conforman tu hogar?,¿Con cuáles de los siguientes servicios de salud cuentas?,¿Con cuáles de los siguientes servicios secundarios cuentas en tu hogar?,¿Con cuáles de estos bienes cuenta tu vivienda? Puedes seleccionar más de uno,¿Cuáles de estos bienes personales posees? Puedes seleccionar más de uno,En caso de que tengas smartphone ¿Qué sistema operativo tiene?,¿Dónde vivirás mientras cursas tus estudios universitarios?,¿De qué forma financiarás tus estudios universitarios?,¿Trabajas actualmente?,¿Cuántas horas trabajas a la semana?,¿Tu trabajo tiene relación con la carrera que estas estudiando?,"Selecciona, según sea el caso, el tipo de institución en la que cursaste los niveles básicos de estudio [Primaria]","Selecciona, según sea el caso, el tipo de institución en la que cursaste los niveles básicos de estudio [Secundaria]",¿En dónde cursaste el bachillerato?,¿Cómo se llama la institución donde cursaste el bachillerato?,¿Qué promedio obtuviste en el bachillerato?,¿En cuántos años lo terminaste?,¿Recibiste alguna beca?,Simultaneo a tus estudios de bachillerato ¿tuviste que trabajar?,Señala tu nivel de conocimiento de idiomas [Inglés],Señala tu nivel de conocimiento de idiomas [Otro idioma],Señala tu nivel de conocimiento de idiomas [Lengua indígena],¿Tienes algún certificado que corrobore tu nivel de inglés?,¿Tienes estudios incompletos de una carrera profesional o técnica superior?,"¿Qué nivel de conocimientos posees de...? [Procesadores de texto (Word, TeX, etc.)]","¿Qué nivel de conocimientos posees de...? [Hojas de cálculo (Excel, Calc, etc.)]",¿Qué nivel de conocimientos posees de...? [Programación],¿Qué nivel de conocimientos posees de...? [Gestor de base de datos (SQL)],¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?,¿Cuál fue el proceso de admisión por el que entraste a esta carrera?,Escoger ésta carrera fue...,Al escoger la FES Acatlán ésta fue:,¿Cuándo tomaste la decisión de lo que querías estudiar?,¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Recomendación de conocidos],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Mucha oferta laboral para esta carrera],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Promoción y prestigio social],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Es una carrera fácil],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Es una carrera corta],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [No me admitieron en otra carrera],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Me obligaron mis papás],¿Cuánta importancia le diste a cada uno de estos factores en tu decisión de estudiar esta carrera? [Me la recomendaron conocidos y amigos],¿Cu

In [ ]:
df = df.loc[df["Edad 🎂"] < 40].reset_index(drop=True)
df = df.loc[df['¿Cuántos cuartos para dormir tiene tu vivienda?']>0].reset_index(drop=True)
df = df.loc[df['¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?']<=100000].reset_index(drop=True)
df = df.loc[df['¿En cuántos años lo terminaste?']<=5].reset_index(drop=True)
df.shape

(222, 79)

In [ ]:
varc = ['Edad 🎂',
        '¿Cuánto tiempo dura (en minutos) el trayecto de tu casa a la facultad? ⏱',
        '¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti',
        '¿Cuántos son hombres?',
        '¿Cuántas son mujeres?',
        '¿Cuánto tiempo llevas viviendo ahí? (años)',
        '¿Cuántos cuartos para dormir tiene tu vivienda?',
        '¿Cuántos automóviles en total tienen las personas que conforman tu hogar?',
        '¿Cuántas horas trabajas a la semana?',
        '¿Qué promedio obtuviste en el bachillerato?',
        '¿En cuántos años lo terminaste?',
        '¿A cuánto aproximadamente ascienden los ingresos de tu hogar(considerando a todos los que aportan)?',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Deportes ⚽️🏀🏈]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades culturales 🎭🎬🎻]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Actividades sociales (fiestas, reuniones con amigos, familia, etc.)]',
        '¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]'
       ]

vard = ['Estado civil','¿Tienes hijos?',
        '¿Tienes alguna dificultad para? [👓 Ver]',
        '¿Tienes alguna dificultad para? [👂 Escuchar]',
        '¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física]',
        '¿Dónde vives actualmente?',
        '¿En qué delegación o municipio?',
        '¿Qué medio de transporte utilizas para llegar a la facultad?',
        '¿Cuál de las siguientes opciones describe tu tipo de vivienda?',
        '¿Con quién vives? (Puedes seleccionar más de una)',
        '¿Cuál es el máximo nivel de estudios de tu papá?',
        '¿Cuál es el máximo nivel de estudios de tu mamá?',
        'En caso de que tengas smartphone ¿Qué sistema operativo tiene?',
        '¿Dónde vivirás mientras cursas tus estudios universitarios?',
        '¿De qué forma financiarás tus estudios universitarios?',
        '¿En dónde cursaste el bachillerato?','¿Cómo se llama la institución donde cursaste el bachillerato?',
        '¿Recibiste alguna beca?','¿Cuál fue el proceso de admisión por el que entraste a esta carrera?',
        'Escoger ésta carrera fue...','Al escoger la FES Acatlán ésta fue:',
        '¿Cuándo tomaste la decisión de lo que querías estudiar?'   
       ]
tgt = 'Carrera'

In [ ]:
df.isnull().sum()

Carrera                                                                                                                                                               0
Género                                                                                                                                                                0
Edad 🎂                                                                                                                                                                0
Estado civil                                                                                                                                                          0
¿Tienes hijos?                                                                                                                                                        0
                                                                                                                                                                

## Normalizando variables

In [ ]:
for v in vard:
    df[v].fillna('MISSING',inplace=True)

In [ ]:
def normalizar(df,v,umbral=0.05):
    aux = df[v].value_counts(True).to_frame()
    aux['norm'] = np.where(aux[v]<umbral,'Otros',aux.index)
    grupo = aux.loc[aux['norm']=='Otros'][v].sum()
    if grupo<umbral:
        aux['norm'].replace({'Otros':aux.index[0]},inplace=True)
    aux.drop(v,axis=1,inplace=True)
    return v,aux.to_dict()['norm']

In [ ]:
l_norm = list(map(lambda v:normalizar(df,v),vard))

In [ ]:
for v,d in l_norm:
    df[f'n_{v}'] = df[v].replace(d)

In [ ]:
varn = df.filter(like='n_').columns.tolist()

In [ ]:
for v in varn:
    print(v,'\n')
    print(df[v].value_counts(True).sort_index())
    print('\n'*2)

n_Estado civil 

👤 Soltero            0.779279
👥 En una relación    0.220721
Name: n_Estado civil, dtype: float64



n_¿Tienes hijos? 

No    1.0
Name: n_¿Tienes hijos?, dtype: float64



n_¿Tienes alguna dificultad para? [👓 Ver] 

No    0.513514
Si    0.486486
Name: n_¿Tienes alguna dificultad para? [👓 Ver], dtype: float64



n_¿Tienes alguna dificultad para? [👂 Escuchar] 

No    1.0
Name: n_¿Tienes alguna dificultad para? [👂 Escuchar], dtype: float64



n_¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física] 

No    1.0
Name: n_¿Tienes alguna dificultad para? [💪 Realizar alguna actividad física], dtype: float64



n_¿Dónde vives actualmente? 

CDMX                0.382883
Estado de México    0.617117
Name: n_¿Dónde vives actualmente?, dtype: float64



n_¿En qué delegación o municipio? 

Naucalpan    0.085586
Otros        0.914414
Name: n_¿En qué delegación o municipio?, dtype: float64



n_¿Qué medio de transporte utilizas para llegar a la facultad? 

Transporte públi

In [ ]:
unarias = [v for v in varn if df[v].value_counts().shape[0]==1]

In [ ]:
varn = [v for v in varn if v not in unarias]

## Discretización de variables continuas

In [ ]:
l = []
for v in varc:
    for st in ['uniform','quantile']:
        for k in range(2,6):
            kb = KBinsDiscretizer(n_bins=k,
                                  strategy=st,
                                  encode='ordinal')
            
            miss, nomiss = df.loc[df[v].isnull()][[v]],df.loc[~df[v].isnull()][[v]]
            kb.fit(nomiss[[v]])
            nombre = f'd_{v}_{k}_{st[:3]}' 
            miss[nombre] = miss[v].fillna('MISSING')
            nomiss[nombre] = pd.cut(nomiss[v],bins=kb.bin_edges_[0],include_lowest=True).astype(str)
            l.append(pd.concat([nomiss[[nombre]],miss[[nombre]]]))

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:202: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/t

In [ ]:
dfd = reduce(lambda x,y:pd.merge(x,y,left_index=True,right_index=True,how='outer'),l)

In [ ]:
dfd.shape,df.shape

((222, 128), (222, 101))

## Juntar discretas, continuas y target

In [ ]:
df = df[[tgt]+varn].merge(dfd,left_index=True,right_index=True,how='outer')

In [ ]:
df[tgt] = (df[tgt] =='Actuaria').astype(int)

In [ ]:
df[tgt].value_counts(1)

1    0.68018
0    0.31982
Name: Carrera, dtype: float64

## Calcular IV

In [ ]:
v = 'n_¿Cuál de las siguientes opciones describe tu tipo de vivienda?'

In [ ]:
def iv(df,v,tgt):
    aux = df[[v,tgt]].assign(n=1)
    aux = aux.pivot_table(index=v,
                    columns=tgt,
                    values='n',
                    fill_value=0,
                    aggfunc='sum')
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] = np.log(aux[0]/aux[1])
    aux['iv'] = (aux[0]-aux[1])*aux['woe']
    return v,aux['iv'].sum()

In [ ]:
var = [v for v in df.columns if v != tgt]

In [ ]:
ivdf = pd.DataFrame(map(lambda v:iv(df,v,tgt),var),columns=['variable','iv'])
ivdf = ivdf.sort_values(by='iv',ascending=False).reset_index(drop=True)
ivdf['iv'].replace({np.inf:np.nan,-np.inf:np.nan},inplace=True)
ivdf = ivdf.dropna().reset_index(drop=True)
ivdf['raiz'] = ivdf['variable'].map(lambda x:x.split('_')[1] if x[:2]=='d_' else x)
ivdf = ivdf.sort_values(by=['raiz','iv'],ascending=[1,0]).reset_index(drop=True)
ivdf['best'] = ivdf.groupby('raiz').cumcount()+1
ivdf = ivdf.loc[ivdf['best']==1].drop('best',axis=1).sort_values(by='iv',ascending=False).reset_index(drop=True)

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site

In [ ]:
best = ivdf.loc[ivdf['iv']>=0.1]['variable'].tolist()
best

['d_¿Qué promedio obtuviste en el bachillerato?_4_qua',
 'n_Escoger ésta carrera fue...',
 'n_¿Cuál fue el proceso de admisión por el que entraste a esta carrera?',
 'n_¿Cuándo tomaste la decisión de lo que querías estudiar?',
 'd_¿En cuántos años lo terminaste?_3_uni',
 'n_¿En dónde cursaste el bachillerato?',
 'd_¿Cuántos días a la semana dedicas tiempo para realizar las siguientes actividades? [Entretenimiento personal (videojuegos, redes sociales, netflix, youtube, etc)]_5_qua',
 'n_¿Recibiste alguna beca?',
 'n_Al escoger la FES Acatlán ésta fue:',
 'n_¿Cómo se llama la institución donde cursaste el bachillerato?',
 'd_¿Cuántas personas en total viven en tu hogar? Incluyéndote a ti_5_qua',
 'd_¿Cuánto tiempo llevas viviendo ahí? (años)_5_qua',
 'n_¿Cuál es el máximo nivel de estudios de tu mamá?',
 'n_¿Cuál es el máximo nivel de estudios de tu papá?']

## Transformación WoE

In [ ]:
X = df[best].copy()
y = df[[tgt]].copy()

In [ ]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

In [ ]:
Xt[tgt] = yt

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
def woe(df,v,tgt):
    aux = df[[v,tgt]].assign(n=1)
    aux = aux.pivot_table(index=v,
                    columns=tgt,
                    values='n',
                    fill_value=0,
                    aggfunc='sum')
    for i in range(2):
        aux[i]/=aux[i].sum()
    aux['woe'] = np.log(aux[0]/aux[1])
    aux.drop(range(2),axis=1,inplace=True)
    return v,aux.to_dict()['woe']
    

In [ ]:
mapa_woe = list(map(lambda v:woe(Xt,v,tgt),best))

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
for v,d in mapa_woe:
    Xt[f'w_{v}'] = Xt[v].replace(d)
    Xv[f'w_{v}'] = Xv[v].replace(d)

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
varw = [v for v in Xt.columns if v[:2]=='w_']

In [ ]:
Xt = Xt[varw]
Xv = Xv[varw]

In [ ]:
aux = pd.concat([Xt,Xv])
aux = aux.describe().T[['min']]
aux['min'] = np.abs(aux['min'])

In [ ]:
quitar = aux.loc[np.isinf(aux['min'])].index.tolist()

In [ ]:
varw = [v for v in varw if v not in quitar]

In [ ]:
Xt = Xt[varw]
Xv = Xv[varw]

## Entrenamiento 

In [ ]:
modelo = LogisticRegression()

In [ ]:
modelo.fit(Xt,yt)

/home/david/Documents/MAC/Septimo/Temas Selectos Est Gustavo/envs/tse/lib/python3.6/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression()

In [ ]:
print(roc_auc_score(y_score=modelo.predict_proba(Xt)[:,1],y_true=yt))
print(roc_auc_score(y_score=modelo.predict_proba(Xv)[:,1],y_true=yv))

0.9131299734748011
0.8651785714285715
